<a href="https://colab.research.google.com/github/Flymshady/traffic_model/blob/main/traffic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import os




In [3]:
tf.__version__

'2.3.0'

In [ ]:
import plotly.io as pio
pio.templates.default = 'plotly_white'